In [ ]:
# !git clone -b state-map https://github.com/tobidairo/Group-2-Data-Visualization.git

In [ ]:
# %cd Group-2-Data-Visualization

In [ ]:
# !pip install dash ngrok pyngrok

In [95]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [96]:
!git config --global user.email "tobidairo@outlook.com"
!git config --global user.name "tobidairo"

In [97]:
!git clone https://github.com/tobidairo/Group-2-Data-Visualization.git

Cloning into 'Group-2-Data-Visualization'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7 (delta 1), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), done.
Resolving deltas: 100% (1/1), done.


In [98]:
%cd Group-2-Data-Visualization

/content/Group-2-Data-Visualization


In [99]:
!git checkout -b state-map-frequency-charts

Switched to a new branch 'state-map-frequency-charts'


In [102]:
!cp /content/drive/MyDrive/Group\ 2\ Data\ Visualization/Colab\ Notebooks/.ipynb /content/Group-2-Data-Visualization

cp: cannot stat '/content/drive/MyDrive/Group 2 Data Visualization/Colab Notebooks/.ipynb': No such file or directory


In [36]:
##############################################################################
# Only run this code block if you are not on a local runtime (e.g. if you are using
# Google Colab). This will give you a public URL to access the Dashboard.
# You may have to creat your own ngrok credentials for this to work.
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token("2kNmHk0jeYiQXmN2eHVhiO5HYMK_3GGARUJWCnV2avBZtQFaz")

# Set up a tunnel on port 8050 (default Dash port)
public_url = ngrok.connect(8050) # This sets up a tunnel on port 8050, the default Dash port
print(f'Public URL: {public_url}')
##############################################################################

Public URL: NgrokTunnel: "https://1ce4-34-19-123-218.ngrok-free.app" -> "http://localhost:8050"


In [67]:
import pandas as pd
import numpy as np
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from pyngrok import ngrok
from threading import Thread
import logging
from io import StringIO
import gdown

In [4]:
file_id = '1ZdsrtNY3H7Oh_ojb3vootMMyV84Kw002'
download_url = f'https://drive.google.com/uc?export=download&id={file_id}'
output = 'data.csv'
gdown.download(download_url, output, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1ZdsrtNY3H7Oh_ojb3vootMMyV84Kw002
From (redirected): https://drive.google.com/uc?export=download&id=1ZdsrtNY3H7Oh_ojb3vootMMyV84Kw002&confirm=t&uuid=0f31aa86-098c-46dd-9397-403c9d42733b
To: /content/data.csv
100%|██████████| 492M/492M [00:11<00:00, 42.1MB/s]


'data.csv'

In [5]:
dtypes = {
    'state': 'int8',
    'employment': 'int8',
    'marital_status': 'int8',
    'cardiac_event': 'int8',
    'stroke': 'int8',
    'mental_health': 'int8',
    'medcost': 'int8',
    'checkup': 'int8',
    'eye_exam': 'int8',
    'physical_health': 'int8',
    'poor_health': 'int8',
    'stop_smoking': 'int8',
    'bmi_category': 'int8',
    'education': 'int8',
    'general_health': 'int8',
    'health_insurance': 'int8',
    'exercise': 'int8',
    'asthma': 'int8',
    'arthritis': 'int8',
    'sex': 'int8',
    'age': 'int8',
    'height': 'float64',
    'weight': 'float64',
    'overweight': 'int8',
    'children': 'int8',
    'income': 'int8',
    'race': 'int8',
    'smoking': 'int8',
    'binge_drinking': 'int8',
    'heavy_drinking': 'int8',
    'flu_jab': 'int8',
    'pneumonia_jab': 'int8',
    'aids_test': 'int8',
    'wt': 'float64'
}

In [6]:
df = pd.read_csv('data.csv', header=0, dtype=dtypes)

In [7]:
state_mapping = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO',
    9: 'CT', 10: 'DE', 11: 'DC', 12: 'FL', 13: 'GA',
    15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS',
    21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD', 25: 'MA',
    26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO', 30: 'MT',
    31: 'NE', 32: 'NV', 33: 'NH', 34: 'NJ', 35: 'NM',
    36: 'NY', 37: 'NC', 38: 'ND', 39: 'OH', 40: 'OK',
    41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC', 46: 'SD',
    47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA',
    54: 'WV', 55: 'WI', 56: 'WY', 66: 'GU', 72: 'PR', 78: 'VI'
}

df['state_code'] = df['state'].map(state_mapping)

In [78]:
# Define the weighted mean function
def weighted_mean(df, value_col, weight_col):
    df = df[df[value_col] != -1]  # Filter out invalid values
    if df.empty or df[weight_col].sum() == 0:
        return np.nan
    return (df[value_col] * df[weight_col]).sum() / df[weight_col].sum()

# Define the weighted median with interpolation function
def weighted_median_interpolated(df, value_col, weight_col='wt'):
    df = df[df[value_col] != -1]  # Filter out invalid values

    if df.empty or df[weight_col].sum() == 0:
        return np.nan  # Handle edge cases

    df = df.sort_values(value_col)  # Sort by category

    cumulative_weight = df[weight_col].cumsum()  # Calculate cumulative weights
    total_weight = df[weight_col].sum()
    cutoff = total_weight / 2.0  # Median cutoff

    idx = cumulative_weight.searchsorted(cutoff)

    if idx == 0:
        return df[value_col].iloc[0] + 0.5  # Start of the first category, assume middle of the range

    lower_value = df[value_col].iloc[idx - 1]
    upper_value = df[value_col].iloc[idx]
    lower_weight = cumulative_weight.iloc[idx - 1]
    upper_weight = cumulative_weight.iloc[idx]

    if cumulative_weight.iloc[idx] == cutoff:
        # If exactly on the cutoff, the median is the midpoint of this category
        return (lower_value + upper_value) / 2.0
    else:
        # Interpolating within the category
        fraction = (cutoff - lower_weight) / (upper_weight - lower_weight)
        return lower_value + fraction

# Define the weighted frequency aggregation function for categorical data
def weighted_frequency(df, value_col, weight_col='wt'):
    return df.groupby(value_col)[weight_col].sum()

def aggregate_weighted_frequency(df, groupby_cols, value_col, weight_col='wt'):
    result = df.groupby(groupby_cols).apply(lambda x: weighted_frequency(x, value_col, weight_col).reset_index())
    result = result.reset_index().drop(columns='level_2')
    result.columns = groupby_cols + [value_col, 'Weighted Frequency']
    return result

# Define the aggregate custom function for continuous data
def aggregate_custom(df, groupby_cols, value_col, result_col_name, agg_func, weight_col='wt', scale_factor=1):
    result = df.groupby(groupby_cols).apply(lambda x: agg_func(x, value_col, weight_col)).reset_index()
    result.columns = groupby_cols + [result_col_name]
    if scale_factor != 1:
        result[result_col_name] = result[result_col_name] / scale_factor
    return result

# Map income and age ranges to their midpoints
income_bracket_midpoints = {
    1: 7500,    # Midpoint of "<$15,000"
    2: 20000,   # Midpoint of "$15,000 - $25,000"
    3: 30000,   # Midpoint of "$25,000 - $35,000"
    4: 42500,   # Midpoint of "$35,000 - $50,000"
    5: 75000,  # Midpoint of "$50,000 - $100,000"
    6: 150000,  # Midpoint of "$100,000 - $200,000"
    7: 300000,  # Estimated midpoint of "$200,000+"
    -1: -1
}

df['income_midpoint'] = df['income'].map(income_bracket_midpoints)

age_range_midpoints = {
    1: 21.5,
    2: 30,
    3: 40,
    4: 50,
    5: 60,
    6: 75,
    -1: -1
}

df['age_midpoint'] = df['age'].map(age_range_midpoints)

# Aggregate data for continuous variables
income_by_year_state = aggregate_custom(
    df,
    groupby_cols=['year', 'state_code'],
    value_col='income_midpoint',
    result_col_name='Average Income',
    agg_func=weighted_mean
)

height_by_year_state = aggregate_custom(
    df,
    groupby_cols=['year', 'state_code'],
    value_col='height',
    result_col_name='Average Height',
    agg_func=weighted_mean
)

weight_by_year_state = aggregate_custom(
    df,
    groupby_cols=['year', 'state_code'],
    value_col='weight',
    result_col_name='Average Weight',
    agg_func=weighted_mean,
    scale_factor=100
)

age_by_year_state = aggregate_custom(
    df,
    groupby_cols=['year', 'state_code'],
    value_col='age_midpoint',
    result_col_name='Average Age',
    agg_func=weighted_mean
)

In [94]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Header(
        html.H1("US Survey Data", style={'text-align': 'center', 'padding': '10px'}),
    ),

    # Variable selector and Year slider
    html.Div(
        [
            html.Div(
                [
                    dcc.RadioItems(
                        id='variable-selector',
                        options=[
                            {'label': 'Income', 'value': 'Average Income'},
                            {'label': 'Height', 'value': 'Average Height'},
                            {'label': 'Weight', 'value': 'Average Weight'},
                            {'label': 'Age', 'value': 'Average Age'}
                        ],
                        value='Average Income',
                        labelStyle={'display': 'inline-block', 'margin-right': '100px',}
                    ),
                ],
                style={'width': '48%', 'display': 'inline-block', 'padding': '20px'}
            ),
            html.Div(
                [
                    dcc.Slider(
                        id='year-slider',
                        min=2012,
                        max=2022,
                        step=1,
                        value=2012,
                        marks={str(year): str(year) for year in range(2012, 2023)},
                    ),
                ],
                style={'width': '48%', 'display': 'inline-block', 'padding': '20px'}
            )
        ],
        style={'display': 'flex', 'justify-content': 'center', 'flex-wrap': 'wrap'}
    ),

    # Graphs layout
    html.Div(
        [
            html.Div(
                dcc.Graph(id='choropleth-map', style={'height': '500px'}),
                style={'width': '48%', 'display': 'inline-block', 'padding': '20px','flex-grow': '1'}
            ),
            html.Div(
                dcc.Graph(id='column-chart', style={'height': '500px'}),
                style={'width': '48%', 'display': 'inline-block', 'padding': '20px', 'flex-grow': '1'}
            ),
        ],
        style={'display': 'flex', 'justify-content': 'space-between', 'align-items': 'center'}
    ),

    html.Div(
    [
        html.Div(
                  [
                      dcc.Dropdown(
                          options=[
                              {'label': 'State', 'value': 'state'},
                              {'label': 'Employment', 'value': 'employment'},
                              {'label': 'Marital Status', 'value': 'marital_status'},
                              {'label': 'Cardiac Event', 'value': 'cardiac_event'},
                              {'label': 'Stroke', 'value': 'stroke'},
                              {'label': 'Mental Health', 'value': 'mental_health'},
                              {'label': 'Medcost', 'value': 'medcost'},
                              {'label': 'Checkup', 'value': 'checkup'},
                              {'label': 'Eye Exam', 'value': 'eye_exam'},
                              {'label': 'Physical Health', 'value': 'physical_health'},
                              {'label': 'Poor Health', 'value': 'poor_health'},
                              {'label': 'Stop Smoking', 'value': 'stop_smoking'},
                              {'label': 'BMI Category', 'value': 'bmi_category'},
                              {'label': 'Education', 'value': 'education'},
                              {'label': 'General Health', 'value': 'general_health'},
                              {'label': 'Health Insurance', 'value': 'health_insurance'},
                              {'label': 'Exercise', 'value': 'exercise'},
                              {'label': 'Asthma', 'value': 'asthma'},
                              {'label': 'Arthritis', 'value': 'arthritis'},
                              {'label': 'Sex', 'value': 'sex'},
                              {'label': 'Age', 'value': 'age'},
                              {'label': 'Height', 'value': 'height'},
                              {'label': 'Weight', 'value': 'weight'},
                              {'label': 'Overweight', 'value': 'overweight'},
                              {'label': 'Children', 'value': 'children'},
                              {'label': 'Income', 'value': 'income'},
                              {'label': 'Race', 'value': 'race'},
                              {'label': 'Smoking', 'value': 'smoking'},
                              {'label': 'Binge Drinking', 'value': 'binge_drinking'},
                              {'label': 'Heavy Drinking', 'value': 'heavy_drinking'},
                              {'label': 'Flu Jab', 'value': 'flu_jab'},
                              {'label': 'Pneumonia Jab', 'value': 'pneumonia_jab'},
                              {'label': 'AIDS Test', 'value': 'aids_test'}
                          ],
                          value='age',
                          placeholder='Select a variable',
                          id='variable-selector-2'
                      ),
                      html.Div(id='dd-output-container-2')
                  ]
              ),
              html.Div(
                  dcc.Graph(id='frequency-chart', style={'height': '500px'}),
                  style={'width': '48%', 'display': 'inline-block', 'padding': '20px', 'flex-grow': '1'}
              )
          ],
          style={'display': 'flex', 'justify-content': 'space-between', 'align-items': 'center'}
      )

])

@app.callback(
    [Output('choropleth-map', 'figure'),
     Output('column-chart', 'figure'),
     Output('dd-output-container-2', 'children'),
     Output('frequency-chart', 'figure')],
    [Input('year-slider', 'value'),
     Input('variable-selector', 'value'),
     Input('variable-selector-2', 'value')]
)
def update_graphs(selected_year, selected_variable, selected_variable_2, df=df):
    # Filter the data for the selected year
    income_data = income_by_year_state[income_by_year_state['year'] == selected_year].drop(columns=['year'])
    height_data = height_by_year_state[height_by_year_state['year'] == selected_year].drop(columns=['year'])
    weight_data = weight_by_year_state[weight_by_year_state['year'] == selected_year].drop(columns=['year'])
    age_data = age_by_year_state[age_by_year_state['year'] == selected_year].drop(columns=['year'])

    # Merge the dataframes on state_code (assuming all dataframes have this common column)
    merged_data = income_data.merge(height_data, on='state_code', how='inner').merge(weight_data, on='state_code', how='inner').merge(age_data, on='state_code', how='inner')

    print(f"Data for the year {selected_year}")
    print(merged_data)

    df_filtered = df[df['year'] == selected_year]
    df_grouped = df_filtered.groupby(selected_variable_2)
    weighted_frequency = df_grouped['wt'].sum().reset_index()
    weighted_frequency.columns = [selected_variable_2, 'weighted_frequency']
    mapping_dict = get_mapping_dict(selected_variable_2)
    weighted_frequency['mapped_labels'] = weighted_frequency[selected_variable_2].map(mapping_dict)

    max_value = weighted_frequency['weighted_frequency'].max()
    max_label = weighted_frequency.loc[weighted_frequency['weighted_frequency'] == max_value, 'mapped_labels'].values[0]

    fixed_ranges = {
    'Average Income': (30000, 80000),  # Example range in dollars
    'Average Height': (160, 175),  # Example range in cm
    'Average Weight': (74, 88),  # Example range in kg
    'Average Age': (42, 52)  # Example range in years
    }

    # Create the choropleth map
    choropleth_map = px.choropleth(
        data_frame=merged_data,
        locationmode='USA-states',
        locations='state_code',
        scope='usa',
        color=selected_variable,
        hover_data={
            'Average Income': ':.2f',
            'Average Height': ':.2f',
            'Average Weight': ':.2f',
            'Average Age': ':.2f'
        },
        color_continuous_scale=px.colors.sequential.Blues,
        range_color=fixed_ranges[selected_variable],
        labels={
            'Average Income': 'Average Income ($)',
            'Average Height': 'Average Height (cm)',
            'Average Weight': 'Average Weight (kg)',
            'Average Age': 'Average Age (years)'
        },
    )

    column_chart = px.bar(
        data_frame=merged_data,
        x='state_code',
        y=selected_variable,
        labels={
            'state_code': 'State',
            selected_variable: selected_variable
        },
        color=selected_variable,
        color_continuous_scale=px.colors.sequential.Inferno_r,
        range_y=fixed_ranges[selected_variable]
    )



    frequency_chart = px.bar(
        weighted_frequency,
        x='mapped_labels',
        y='weighted_frequency',
        color='weighted_frequency',
        color_continuous_scale=px.colors.sequential.PuBuGn,
        title=f'<b>Adult population count by {selected_variable_2} in {selected_year}</b>',
        labels={
            'mapped_labels': selected_variable_2,
            'weighted_frequency': 'Count'
        },
        template='plotly',
        hover_data={'weighted_frequency': ':.2f'},
        text='weighted_frequency'
    )

    # Code to enhance chart layout
    frequency_chart.update_layout(
        title_font_size=24,  # Increase title font size
        xaxis_title=f'<b>{selected_variable_2}</b>',  # Bold x-axis label
        yaxis_title='<b>Count</b>',  # Bold y-axis label
        margin=dict(t=50, b=50, l=50, r=50),  # Adjust margins
        coloraxis_colorbar=dict(
            title='Count',
            title_font_size=16,
            tickvals=[0, max(weighted_frequency['weighted_frequency'])//2, max(weighted_frequency['weighted_frequency'])],
            ticktext=['Low', 'Medium', 'High']  # Add colorbar tick text
        )
    )

    # Add annotations for key points
    frequency_chart.add_annotation(
        x=max_label,  # x value of max
        y=max_value,  # y value of max
        text="Highest Count",  # Annotation text
        showarrow=True,
        arrowhead=2,
        ax=-50,
        ay=-50
    )

    # Update hover mode
    frequency_chart.update_traces(
        hovertemplate='<b>%{x}</b><br>Count: %{y}<extra></extra>',
        textposition='outside',
        texttemplate='%{text:.2s}',
    )



    return choropleth_map, column_chart, f"Selected variable: {selected_variable_2}", frequency_chart

def run_dash_app():
    app.run_server(port=8050, debug=True)

import threading
thread = threading.Thread(target=run_dash_app)
thread.start()

def get_mapping_dict(selected_variable):
    if selected_variable == 'state':
        return {
            1: 'Alabama', 2: 'Alaska', 4: 'Arizona', 5: 'Arkansas', 6: 'California', 8: 'Colorado',
            9: 'Connecticut', 10: 'Delaware', 11: 'District of Columbia', 12: 'Florida', 13: 'Georgia',
            15: 'Hawaii', 16: 'Idaho', 17: 'Illinois', 18: 'Indiana', 19: 'Iowa', 20: 'Kansas',
            21: 'Kentucky', 22: 'Louisiana', 23: 'Maine', 24: 'Maryland', 25: 'Massachusetts',
            26: 'Michigan', 27: 'Minnesota', 28: 'Mississippi', 29: 'Missouri', 30: 'Montana',
            31: 'Nebraska', 32: 'Nevada', 33: 'New Hampshire', 34: 'New Jersey', 35: 'New Mexico',
            36: 'New York', 37: 'North Carolina', 38: 'North Dakota', 39: 'Ohio', 40: 'Oklahoma',
            41: 'Oregon', 42: 'Pennsylvania', 44: 'Rhode Island', 45: 'South Carolina', 46: 'South Dakota',
            47: 'Tennessee', 48: 'Texas', 49: 'Utah', 50: 'Vermont', 51: 'Virginia', 53: 'Washington',
            54: 'West Virginia', 55: 'Wisconsin', 56: 'Wyoming', 66: 'Guam', 72: 'Puerto Rico', 78: 'Virgin Islands'
        }

    elif selected_variable == 'employment':
        return {
            1: 'Employed',
            2: 'Unemployed',
            3: 'Economically Inactive',
            -1: 'Other'
        }

    elif selected_variable == 'marital_status':
        return {
            1: 'Currently Married',
            2: 'Previously Married',
            3: 'Never Married',
            -1: 'Other'
        }

    elif selected_variable == 'cardiac_event':
        return {
            1: 'Had a heart attack / angina / CHD before',
            2: 'Never had before',
            -1: 'Other'
        }

    elif selected_variable == 'stroke':
        return {
            1: 'Had a stroke before',
            2: 'Never had a stroke before',
            -1: 'Other'
        }

    elif selected_variable == 'mental_health':
        return {
            1: 'Zero days when mental health was not good',
            2: '1-13 days when mental health was not good',
            3: '14+ days when mental health was not good',
            -1: 'Other'
        }

    elif selected_variable == 'medcost':
        return {
            1: 'Couldn’t afford to see doctor in last 12 months',
            2: 'Could afford to see doctor in last 12 months',
            -1: 'Other'
        }

    elif selected_variable == 'checkup':
        return {
            1: 'Within past year',
            2: 'Between 1 and 2 years ago',
            3: 'Between 2 and 5 years ago',
            4: '5 or more years ago',
            8: 'Never',
            -1: 'Other'
        }

    elif selected_variable == 'eye_exam':
        return {
            1: 'Within past month',
            2: 'Past year',
            3: 'Between 1 and 2 years ago',
            4: '2 or more years ago',
            8: 'Never',
            -1: 'Other'
        }

    elif selected_variable == 'physical_health':
        return {
            1: 'Zero days when physical health was not good',
            2: '1-13 days when physical health was not good',
            3: '14+ days when physical health was not good',
            -1: 'Other'
        }

    elif selected_variable == 'poor_health':
        return {
            1: 'Zero days',
            2: '1-13 days',
            3: '14+ days',
            -1: 'Other'
        }

    elif selected_variable == 'stop_smoking':
        return {
            1: 'Attempted to quit smoking in past 12 months',
            2: 'Have not attempted',
            -1: 'Other'
        }

    elif selected_variable == 'bmi_category':
        return {
            1: 'Underweight',
            2: 'Normal weight',
            3: 'Overweight',
            4: 'Obese',
            -1: 'Other'
        }

    elif selected_variable == 'education':
        return {
            1: 'Did not graduate high school',
            2: 'Graduated high school',
            3: 'Attended college or technical school',
            4: 'Graduated from college or technical school',
            -1: 'Other'
        }

    elif selected_variable == 'general_health':
        return {
            1: 'Good or better health',
            2: 'Fair or poor health',
            -1: 'Other'
        }

    elif selected_variable == 'health_insurance':
        return {
            1: 'Have some form of health insurance',
            2: 'Do not have any form of health insurance',
            -1: 'Other'
        }

    elif selected_variable == 'exercise':
        return {
            1: 'Had physical activity or exercise',
            2: 'No physical activity or exercise in last 30 days',
            -1: 'Other'
        }

    elif selected_variable == 'asthma':
        return {
            1: 'Current',
            2: 'Former',
            3: 'Never',
            -1: 'Other'
        }

    elif selected_variable == 'arthritis':
        return {
            1: 'Diagnosed',
            2: 'Not diagnosed',
            -1: 'Other'
        }

    elif selected_variable == 'sex':
        return {
            1: 'Male',
            2: 'Female',
            -1: 'Other'
        }

    elif selected_variable == 'age':
        return {
            1: 'Age 18-24',
            2: 'Age 25-34',
            3: 'Age 45-54',
            4: 'Age 55-64',
            6: 'Age 65 or older',
            -1: 'Other'
        }

    elif selected_variable == 'height':
        return {}  # continuous variable

    elif selected_variable == 'weight':
        return {}  # continuous variable

    elif selected_variable == 'overweight':
        return {
            1: 'Not overweight or obese',
            2: 'Overweight or obese',
            -1: 'Other'
        }

    elif selected_variable == 'children':
        return {
            1: 'No children in household',
            2: 'One child in household',
            3: '2 children',
            4: '3 children',
            5: '4 children',
            6: '5 or more children',
            -1: 'Other'
        }

    elif selected_variable == 'income': # this mapping is currently only correct for 2021 and 2022
        return {
            1: 'Less than $15,000',
            2: '$15,000 - $25,000',
            3: '$25,000 - $35,000',
            4: '$35,000 - $50,000',
            5: '$50,000 - $100,000',
            6: '$100,000 - $200,000',
            7: '$200,000 or more',
            -1: 'Other'
        }

    elif selected_variable == 'race':
        return {
            1: 'White',
            2: 'Black or African American',
            3: 'American Indian or Alaskan Native',
            4: 'Asian',
            5: 'Native Hawaiian or Other Pacific Islander',
            6: 'Multiracial',
            -1: 'Other'
        }

    elif selected_variable == 'smoking':
        return {
            1: 'Current smoker - now every day',
            2: 'Current smoker - now some days',
            3: 'Former smoker',
            4: 'Never smoked',
            -1: 'Other'
        }

    elif selected_variable == 'binge_drinking':
        return {
            1: 'Did not binge drink in past 30 days',
            2: 'Did binge drink in past 30 days',
            -1: 'Other'
        }

    elif selected_variable == 'heavy_drinking':
        return {
            1: 'Not a heavy drinker',
            2: 'A heavy drinker',
            -1: 'Other'
        }

    elif selected_variable == 'flu_jab':
        return {
            1: 'Over 65 who have had a flu jab in the past year',
            2: 'Over 65 who have not had a flu jab in past year',
            -1: 'Other'
        }

    elif selected_variable == 'pneumonia_jab':
        return {
            1: 'Over 65 who have had a pneumonia jab in past year',
            2: 'Over 65 who have not had a pneumonia jab in past year',
            -1: 'Other'
        }

    elif selected_variable == 'aids_test':
        return {
            1: 'Have been tested for HIV',
            2: 'Have not been tested for HIV',
            -1: 'Other'
        }

    else:
        return {-1: 'Other'}
